In [1]:
import feedparser
podcast_feed_url = "https://access.acast.com/rss/d556eb54-6160-4c85-95f4-47d9f5216c49"
podcast_feed = feedparser.parse(podcast_feed_url)

In [2]:
print ("The number of podcast entries is ", len(podcast_feed.entries))

The number of podcast entries is  1601


In [3]:
import os
import requests


# Ensure the download directory exists
download_dir = "./content/audio"
os.makedirs(download_dir, exist_ok=True)


episode_url = podcast_feed.entries[0].enclosures[0].href
response = requests.get(episode_url, stream=True)

if response.status_code == 200:
    with open("./content/audio/podcast_episode1.mp3", "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print("Download complete!")
else:
    print("Failed to download:", response.status_code)


Download complete!


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
%%time

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cuda:0


CPU times: user 6.44 s, sys: 1.18 s, total: 7.63 s
Wall time: 12.5 s


/home/teng/development/projects/public/podcast-audio-transcription/podcastenv/lib/python3.10/site-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


In [6]:
%%time

result = pipe("./content/audio/podcast_episode1.mp3")
print(result["text"])

/home/teng/development/projects/public/podcast-audio-transcription/podcastenv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Part of Donald Trump's rapidly shifting policy on Ukraine is the claim that America has spent far more money than other allies have. Notably, Europe. We dig into the numbers. spoiler alert, he's wrong. And if you had a relatively distinguished career, have good personal and communication skills, and fancy a real change, Britain's Parliament may have an offer for you. Be warned, though, you're going to have to wear tights. First up though, though. Some of what happened overnight in Germany went entirely as expected. The Christian Democratic Union and its sister Christian Social Union topped the poll with 29%. The CDU's leader Friedrich Merz will be Chancellor. As expected, the hard-right alternative for Germany or AFD party had its best ever national result by a factor of 2, 21% of the vote. A bit more surpri

In [7]:
# Check the transcription happened correctly by peeking into the first 1000 characters
podcast_transcript = result['text']
result['text'][:1000]

" The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Part of Donald Trump's rapidly shifting policy on Ukraine is the claim that America has spent far more money than other allies have. Notably, Europe. We dig into the numbers. spoiler alert, he's wrong. And if you had a relatively distinguished career, have good personal and communication skills, and fancy a real change, Britain's Parliament may have an offer for you. Be warned, though, you're going to have to wear tights. First up though, though. Some of what happened overnight in Germany went entirely as expected. The Christian Democratic Union and its sister Christian Social Union topped the poll with 29%. The CDU's leader Friedrich Merz will be Chancellor. As expected, the hard-right alternative for Germany or AFD party had its best ever national result by a factor of 2, 21% of the vote. A bit more surpr

In [8]:
podcast_transcript

" The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Part of Donald Trump's rapidly shifting policy on Ukraine is the claim that America has spent far more money than other allies have. Notably, Europe. We dig into the numbers. spoiler alert, he's wrong. And if you had a relatively distinguished career, have good personal and communication skills, and fancy a real change, Britain's Parliament may have an offer for you. Be warned, though, you're going to have to wear tights. First up though, though. Some of what happened overnight in Germany went entirely as expected. The Christian Democratic Union and its sister Christian Social Union topped the poll with 29%. The CDU's leader Friedrich Merz will be Chancellor. As expected, the hard-right alternative for Germany or AFD party had its best ever national result by a factor of 2, 21% of the vote. A bit more surpr

In [9]:
podcast_transcript = """
  The Economist. Hello and welcome to the Intelligence from The Economist. I'm your host Jason Palmer. Every weekday we provide a fresh perspective on the events shaping your world. Part of Donald Trump's rapidly shifting policy on Ukraine is the claim that America has spent far more money than other allies have. Notably, Europe. We dig into the numbers. spoiler alert, he's wrong. And if you had a relatively distinguished career, have good personal and communication skills, and fancy a real change, Britain's Parliament may have an offer for you. Be warned, though, you're going to have to wear tights. First up though, though. Some of what happened overnight in Germany went entirely as expected. The Christian Democratic Union and its sister Christian Social Union topped the poll with 29%. The CDU's leader Friedrich Merz will be Chancellor. As expected, the hard-right alternative for Germany or AFD party had its best ever national result by a factor of 2, 21% of the vote. A bit more surprising was the poor showing of the Social Democrats or SDP which led the outgoing government. That is a bitter swale gapness for the Social Democratic Party. Olaf Schulz, outgoing German Chancellor, called it a bitter election for the party. Ozzo, on top we've got the CDU, CSU alliance, then the AFD, which no one will work with, and then the SDP. You with me so far? Then there are the smaller parties that probably won't end up in government. So as confusing as it all is, our Berlin Bureau Chief Tom Nottle says it's not as messy as it could have been. Well we have woken up to clarity. Frinkink Metz is CDU, my sister party, the Christian Social Union, will lead the next government. They will attempt to form a coalition with the Social Democrats. We had a bit of a nail-bitter overnight when one small party, the Saravagun Connect alliance, was bobbing above and below the 5% threshold needed to enter Parliament. If they had made it, then Mr. Meats would have needed three parties to form a coalition and nobody wanted that in the end the Saravark Connect Alliance fell short and so these two parties will have the numbers to form a coalition. Now when we spoke to you last week it was a riot of colors and possible flags and it all came down to which parties actually made the 5% cut in the election. Talk us through who did and who didn't and what that means for you. Okay, so we're going to have basically a five party Bunders tag. The social democrats who lead the outgoing minority government slumped to a huge defeat on around 16% their worst result in history. They will be needed to make up the numbers in the coalition that the Christian Democrats will lead. The three Democrats, a pro-business party who used to be in the government, they got booted out of Parliament entirely, They didn't make the 5% threshold. Christian Linna, the former finance minister and their leader has said he's going to quit politics. And then finally, the Green Party, who were the junior partner in the outgoing government, pretty bad night for them, slightly down on their result at the last election. They're now on around 12%, and they will now go into opposition as well. In summary of all of that, what does the who's in, who's out kind of tell you about the political mood in Germany right now? I mean essentially Germans are very, very grumpy. One reason I think that the CDUCS you didn't do as well as might have been expected is that Frisieck Matz, the candidate for Chancellor who will now take over from Olav Schwartz as Chancellor is not a popular figure amongst Germans his approval rating sits at around 30% which is pretty bad for someone hoping to take over the government. The problem is that the parties of the outgoing government were even more unpopular. Olaf Schultz's government was staggeringly unpopular. It argued about everything. It couldn't get anything done in Europe. It couldn't get much done at home. It fell apart early in November when the free Democrats were kicked out. That's what precipitated this early election. So you find all manner of polls that find that Germans are very distrustful of politics, extremely grumpy. and of course that's the mood that in large part has lifted the AFD to this extraordinary result. And so about the question of what is in the new Chancellor's intray, the inevitable question. Let's talk about the frustrations that he'll face in particular at home what do you think will be on his mind? Okay three things the first is Germany's flatlining economy in real terms it hasn't grown for six years the second is irregular migration which was perhaps the dominant theme of the election campaign. You might remember that Fridget Meats caused up for last month when he pushed through some anti-immigration motions with votes from the AFD. He wants to turn back asylum seekers from Germany's borders. He wants permanent controls at those borders. Now the SPD with whom he's going to have to begin coalition talks say that these proposals violate German constitution and violate EU law so it's not easy to see how you're going to find a path through that thicket. And the third thing is Germany's legendary debt break. This is the rule in the Constitution that limits the federal government's ability to run deficit. there's a general consensus now that this rule is going to have to be relaxed to allow for more public investment and in particular to allow Germany to raise its defense spending. The problem is that to change the debt rate because it's in the Constitution you need a two-thirds majority in Parliament and the mainstream parties in the next parliament are going to fall slightly short of that. That means that they are going to need some votes from Delinker, a hard left party that did very well yesterday and that is going to involve all sorts of complicated horse trading and compromises that might be very very difficult to achieve. And at the mention of defense spending, what about the international sphere? What will Mr. Merds be thinking about there? So this was one of the most interesting things about last night, Jason. So a few hours after the exit polls appeared, Frisjic Metz, who's being interviewed along with the other party leaders, made some extraordinary statements, particularly about America. The absolute priority of my sake is that the European for standing, that the one cent on the other side, find theDorched in Stadt totally also with the Wall-Stututune and also that's the situation and also these are sufficen versa. He said that Europe was going to have to consider an independent defense capability, independent from NATO. He said that America was not a reliable partner. He said that America's government does not care much about the fate of Europe and to understand just how extraordinary these statements are coming from somebody like Frijic Mez, he is steeped in the transatlanticism of the CDU. In his long years outside of politics he was head of a pro-America lobby group called the Atlantic Bridge. What he appears to understand is that Donald Trump has turned the assumptions about the global diplomatic order on their head. He is now supporting the Russian side rather than the Ukrainian side and this has huge implications for Germany and for Europe. Now normally in Germany after an election the country sort of goes into hibernation when it spends weeks or months negotiating a coalition and it's still going to take time to do that this time around but what Mr Meers appears to understand in fact he said this yesterday is that the world cannot wait for Germany he wants very speedy coalition talks He said he wants to get them concluded by April 20th by Easter. That's optimistic, but there is a sense of urgency that attaches to these coalition talks that I haven't previously seen and for someone like Frijic Meats to make statements like this is a very strong signal that the future of German diplomacy may look very different from what has been in the past. And we spoke last week about in particular the AFD, the firewall, the reluctance of any party to really work with the AFD and yet they had their best result ever. I mean what does that tell you about sort of unresolved tensions inside the electorate? There's no sugar coating this. It's really worrying. The AFD has doubled its score relative to 2021. It's now going to occupy nearly one quarter of the seats in the Bunders tag, and in particular particular had a very good night across eastern Germany. What Fridimert says he will not work with the AFD, he could not have been clearer on that, but it's going to be tricky. And again, these are these two issues, the economy and irregular immigration. Mr. Merritt says, fix those problems and you can deal with the rise of the AFD. That's easier said than done, and in particular if we do end up with a coalition deal between the CDSU and the SPD that is a sort of muddy compromise rather than a clear document, One thing you can say is that a very large majority of Germans continue to reject the AFD, but yes, there's no getting away from it. This is going to be one core domestic challenge for the next German government is trying to deal with the rise of a populist right party that in some respects rejects the entire post-war democratic order that Germany has been built on. Tom, thanks very much for joining us. Thanks, Jason. Tri Rokis Prote. This morning, three years precisely into Russia's invasion of Ukraine President Vladimir Zelensky released a video paying tribute to Ukrainians. Three years of resistance, three years of support, three years of unity. But outside the country the unity of support for the resistance has frayed. Friedrich Merz soon to be Germany's Chancellor, knows this all too well in his extraordinary comments overnight about independence from America. Because among Donald Trump's fixations on falsehoods is the notion of who has ponied up more support for Ukraine. I think Europe has given a hundred billion and we've given let's say 300 plus and it's more important for them. This hints at yet more uncertainty from America about its aid. And for Ukraine, the constancy of support is nearly as important as the magnitude of it. But let's just be clear about which countries have done and are doing the most. Donald Trump has consistently claimed that America has given Ukraine far more than the Europeans have. Anton LaGuardia is the economist's diplomatic editor. He claims America has spent three times more on Ukraine than the Europeans have, but the evidence is that that's completely wrong. Well, I mean, let's dig into the evidence then. What numbers should we be working from here? The place to start is the regular reports done by the Kiel Institute. This is a German think tank that compiles all donations to Ukraine of military and economic aid and crunches the numbers. And the last report that came out earlier in February says that worldwide countries have provided 267 billion euros. That's about 280 billion dollars. The global total is smaller than Donald Trump claims the American contribution has been thus far. America obviously remains a single most important donor but it's also the largest economy but European contributions now outstrip America's aid to Ukraine and regardless of how you count that. Whereby how you count means which kind of aid I guess it's not just pallets of cash showing up in Ukraine? No it's not pallets of cash although there is a lot of cash going to Ukraine it is also equipment that's come from the stores of countries that bought them years if not decades ago and has to be valued. But the main distinction that the Kiel Institute makes is counting aid according to whether it's been already allocated, which in their language means given or specifically earmarked for a particular project or contract and aid that is committed but not yet allocated. And so calculated narrowly as allocated aid, the Europeans have provided about 132 billion euros and the Americans have provided about 114 billion euros. Within that number the Americans are slightly ahead on military aid to Ukraine. If you then count in the aid yet to be allocated, in other words committed but not yet allocated or given, then the numbers are much different. The Europeans are way ahead on both military and financial aid to Ukraine. It's important to note, however, that even within those military numbers, the most crucial weapons have come from America so long-range strike missiles and so on that the ones that make a real difference to the battlefield tend to be American weapons although the Europeans have also provided a small number of cruise missiles. And so far we've been discussing the contributions from Europe as a block, as a continent. What do we know about aid from individual countries yes so the numbers I gave you include both European countries and European institutions as go, the Germans are the biggest single European donor, followed by countries such as Britain and Denmark and others. That's calculated in absolute terms. If you then calculate in terms of share of GDP, in other words how much of a national effort has each country made or the citizens of each country made, then the Baltic states and the Nordic states and Poland come out way on top of everybody else. So in other words countries essentially that are closer to the border with Russia? Yes, the closer you are to Russia, the more you feel the military threat and the more willing you are to help Ukraine fight. This is broadly speaking, but a striking exception is perhaps Japan which is on the other side of the world from Ukraine but has been giving more as a share of GDP than several European countries. And the reason is, number one, they're actually quite close to Russia, and they're worried about Russia, but important Russia and China are forming an increasingly close partnership and the Japanese are extremely worried about China and they feel that they have a direct stake in what happens in Ukraine because that will affect the balance of power in the Pacific. And whatever the numbers for what America has done before now and more or less than the EU on various scores. It seems the direction of travel here is very clearly towards America doing less. That's absolutely right. At the most optimistic end, the Americans will do less to help Ukraine and less to help Europe. Pete Hexeth, the defense secretary made that extremely clear. He said we have other priorities particularly in the Pacific. The pessimistic scenario is that the Americans stop supporting Ukraine altogether and possibly withdraw their forces from Europe where there are tens of thousands of American soldiers. So the Europeans will have to spend a lot more money on their military forces not only to help Ukraine but also to defend themselves. I guess the question there is like is it all hands to the pump if America pulls out like 100% just gone sorry we're busy and Europe is left like basically holding the bag does it feel as a block or any individual country like it has to like step up in a massive way, right? Like what does happen in that scenario? I know this is like the squillion dollar question, but like do you have a thought? Well the rhetoric from the Europeans at the moment is very much we will continue helping Ukraine even if the Americans don't and we will spend more in defense because we must defend ourselves from Russia especially if the Americans pull out, but also in the hope that being seen to be more Marshall will give the friends that they still have in America, an argument with which to say no America must remain committed in some way within Europe even if it's in a lesser fashion than hitherto. So defense spending has both a political signal to Russia and the United States and is a necessity. The question at home is whether they convince their publics to spend more in defense and therefore spend less on social services and other forms of spending at a time when budgets are all under strain. So in short, we're going to have to wait and see what the American administration actually does rather than what it's said so far it's going to do. Yes, you never know exactly what Donald Trump is going to do. So the more cool-headed people in Europe are saying we still have to try and keep the Americans on board. We have to show that we've got more skin in the game, we've got to show that we're doing more in defense. But there's no guarantee that that's going to work. Anton, thanks very much for joining us. I'm pleasure to talk to you. On the British government's website a curious job posting has turned up. Wanted, a lady or gentleman usher of the Black Rod. It's an administrative role and a very ceremonial one. There's been a person known simply as Black Rod in the Palace of Westminster since 1350. I think it's fair to call it an interesting gig. They have to wear tights and the most important part is they carry a black rod which is by the way formerly called a staff. Love this country. Blackrod is a ridiculous title for what is actually very necessary job. Catherine Nixi writes about Britain for the economist. Britain has both a king and a parliament but like oil and water the two don't mix but once a year for the king's speech they have to. Blackrod is the person whose job it is to bring the two together. Black Rod! What Parliament does is they shut the door in the face of Blackrod ceremonyally. And then Blackrod has to knock on go and see the King. But it's a big dramatic show, which is put on really to show that the king does not get to rule the government, that the government is in charge if they do turn up a mauch the king give a speech on a nice fancy throne and so now apparently there's going to be a new black rod in town tell me about the job posting what are they looking for well the job posting has been really clear apart from banging a door with a stick while you're wearing tights at the house of lord says that the candidate must demonstrate strong interpersonal and influencing skills. They have to have a calm manner and they have to have excellent communication skills. This is actually sort of true because Blackrod doesn't merely attend the state opening of parliament. They helped to organize it and lots of other ceremonial events as well. When the Queen was lying in state, Blackrod was one of the people who organized all of that. And they have other duties too. So Blackrod is also an usher of the noble order of the garter. They're the sergeant at arms in the House of Lords. And they are also the secretary to the great Lord Chamberlain, which is a ridiculous series of titles that sounds more like something you might read in a novel by Tolkien than a job advert. Tell me about the sort of the particulars of the job. What are the parks? Well, the salary has caused a bit of a Ferrari because it's 87,000 to 114,000 pounds a year, which everyone seems to think is quite a lot for hitting a door with a stick. And they also have generous civil service pension schemes and they have kind of other perks so they get to have their own corner office which is called Black Rod's Bogs and they have a frankly unparalleled ability to wear tights and ruffles in public. And so what kind of person really wants to tap into that ability to wear tights in public? I think someone who fancies a nice retirement with a few perks so it sounds like a silly job it's been held by some very serious people the current black rod is called Sarah Clark and she ran Wimbledon before she ran the Lords. And all the rods before her have been men. Most of them have been military men, and that's important because one of the other duties of Black Rod is organizing security in the House of Lords. And it's quite a traditional role, but apparently, according to the advert, the hiring process is not going to be. The House of Lords, like the House of Commons, says it's committed to becoming an even more welcoming, inclusive and diverse organization. I mean, if they're really going to stick to that, then I am tempted to apply myself. Are you? Should we give it a go? Do we have a chance, Catherine? So what I spoke to an insider who said that like one of the key things is so one of the good things about military people is that they're really good at organizing the security and one of the bad things about military men is that hairy legs he says can play havoc with the types that you have to wear. So I guess you know if, if you've got the legs for it, I'd go for it. I am, I'm not incredibly her suit. I think I could probably get away with that. I mean, for that kind of money I might even shave my legs. Who wax them for that kind of money? Catherine thanks very much I've got to go fill out an application. Good luck. That's all for this episode of the Intelligence. We'll see you back here tomorrow. you're going to be.
"""

In [10]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"]
)

In [12]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o-mini")
print ("Number of tokens in input prompt ", len(enc.encode(podcast_transcript)))

KeyboardInterrupt: 

In [13]:
instructPrompt = """
You are an expert copywriter who is responsible for publishing newsletters with thousands of subscribers. You recently listened to a great podcast
and want to share a summary of it with your readers. Please write the summary of this podcast making sure to cover the important aspects that were
discussed and please keep it concise.
The transcript of the podcast is provided below"""

request = instructPrompt + podcast_transcript

In [14]:
chatOutput = client.chat.completions.create(model="gpt-4o-mini",
                                            messages=[{"role": "system", "content": "You are a helpful assistant."},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )

In [15]:
podcastSummary = chatOutput.choices[0].message.content
podcastSummary

'**Podcast Summary: The Intelligence from The Economist**\n\nIn a recent episode of "The Intelligence," host Jason Palmer discusses key political developments in Europe and the U.S., as well as the unexpected job opportunity within Britain’s Parliament. \n\n1. **German Elections:** The podcast highlights Germany’s recent election results, with the Christian Democratic Union (CDU) and its sister party, the Christian Social Union (CSU), securing 29% of the vote, leading to Friedrich Merz becoming Chancellor. The hard-right Alternative for Germany (AFD) achieved a record 21% while the Social Democrats (SPD), who led the previous government, suffered their worst results in history at just 16%. The discussion emphasizes political discontent among Germans, where a lack of trust in the government has fueled the AFD\'s rise.\n\n2. **Challenges Ahead:** Merz faces significant challenges, including a flatlining economy, irregular migration, and the need to reform Germany\'s debt regulations to e

In [16]:
print(podcastSummary)

**Podcast Summary: The Intelligence from The Economist**

In a recent episode of "The Intelligence," host Jason Palmer discusses key political developments in Europe and the U.S., as well as the unexpected job opportunity within Britain’s Parliament. 

1. **German Elections:** The podcast highlights Germany’s recent election results, with the Christian Democratic Union (CDU) and its sister party, the Christian Social Union (CSU), securing 29% of the vote, leading to Friedrich Merz becoming Chancellor. The hard-right Alternative for Germany (AFD) achieved a record 21% while the Social Democrats (SPD), who led the previous government, suffered their worst results in history at just 16%. The discussion emphasizes political discontent among Germans, where a lack of trust in the government has fueled the AFD's rise.

2. **Challenges Ahead:** Merz faces significant challenges, including a flatlining economy, irregular migration, and the need to reform Germany's debt regulations to enable inc